# Notes

### `{{cookiecutter.project_name}}::{{cookiecutter.session_id}}`

## 1. Usage

### 1.1. Jupyter

*You can fill inn the MarkDown cells (the cells without "numbering") by double-clicking them. Also remember, press `shift + enter` to execute a cell.*

A couple of useful links:  
 - [How to write MarkDown](https://github.com/adam-p/markdown-here/wiki/Markdown-Cheatsheet#hr)
 - [Jupyter notebooks](https://jupyter.org/)
 - [cellpy](https://cellpy.readthedocs.io/en/latest/)

### 1.2. The `cellpy` notebooks

This set of notebooks is intended to give you a structure for performing a proper `cellpy` analysis of your data utilising the `cellpy.utils.bathc` tool.

An example of a project structure can be this:

```bash

cellpy_project/
└── experiment_001
    ├── batch_file.json
    ├── data
    │   ├── external
    │   ├── interim
    │   ├── processed
    │   └── raw
    ├── out
    │   └── note.md
    ├── notebook: 00_experiment_001_notes.ipynb
    ├── notebook: 01_experiment_001_loader.ipynb
    ├── notebook: 02_experiment_001_life.ipynb
    ├── notebook: 03_experiment_001_cycles.ipynb
    ├── notebook: 04_experiment_001_ica.ipynb
    └── notebook: 05_experiment_001_plots.ipynb

```

The `00_experiment_001_notes.ipynb` notebook can be used for writing a log, as well as giving you some information on how to do stuff.

The `01_experiment_001_loader.ipynb` notebook is the one you use to load the data from the tester(s) and create cellpy-files. You should run this first. There are several options on how to store the cellpy-files and the journal, so chose the option that suits you best. You only have to re-run this notebook if your data needs to be updated.

The next three notebooks use the cellpy-files produced by `01_experiment_001_loader.ipynb`. Try to keep them "understandable" and well structured so that the future you dont get totally lost trying to re-run them.

The last notebook is ment to contain important plots that you would like to be able to use later on. They should preferably be "stand-alone" and not rely on running the other notebooks (as a helpful gesture to the future you).

## 2. Key information about the current experiment

**Experimental-id:** `{{cookiecutter.notebook_name}}`  
**Short-name:** `{{cookiecutter.session_id}}`  
**Project:** `{{cookiecutter.project_name}}`  
**By:** `{{cookiecutter.author_name}}`   
**Date:** `{{cookiecutter.date}}`  

### Notebooks
- notes and information [link](00_{{cookiecutter.notebook_name}}_notes.ipynb)
- processing raw data [link](01_{{cookiecutter.notebook_name}}_loader.ipynb)
- life [link](02_{{cookiecutter.notebook_name}}_life.ipynb)
- cycles [link](03_{{cookiecutter.notebook_name}}_cycles.ipynb)
- ica [link](04_{{cookiecutter.notebook_name}}_ica.ipynb)
- plots [link](05_{{cookiecutter.notebook_name}}_plots.ipynb)

## 3. Short summary of the experiment before processing
It is often helpful to formulate what you wanted to achieve with your experiment before actually going into depth of the data. I believe that it does not make you "biased" when processing your data, but instead sharpens your mind and motivates you to look more closely on your results. I might be wrong, off course. Then just skip filling in this part.

### Main purpose
(*State the main hypothesis for the current set of experiment*)



### Expected outcome
(*What do you expect to find out? What kind of tests did you perform?*)






### Special considerations
(*State if there are any special considerations for this experiment*)





## 4. Log
(*Here you should fill inn what you have done during your analysis*)








## 5. Summary
It is always helpful to formulate what you learned from your experiment. It is very unlikely that I am wrong here. So, please, don't skip filling in this part.

(*What did you discover?*)

(*What should you do further?*)


## Appendix A - Tips and tricks

### Trick: Use magic to automatically reload modules
This allows you to modify modules (and scripts) in another editor and see the changes within Jupyter without shutting down the kernel.

```python
# in the first notebook cell
%load_ext autoreload
%autoreload 2
```

### Example: Average nominal capacity and mass calculations

```python
csf = helpers.concatenate_summaries(b)  # multi-index dataframe
filter_charge_mah = csf.columns.get_level_values(1) == "Charge_Capacity(mAh/g)"
scsf = csf.loc[:, filter_charge_mah]

scsf.columns = scsf.columns.droplevel(1)

df = pd.DataFrame(scsf.mean(), columns=["nom_cap_mAh_g"])

df["mass_old"] = b.pages.masses

nom_cap = 350.0
df["adjusted_mass"]  = df.capacity_old * df.mass_old / nom_cap

```

### Example: Filtering concatenated summaries

```python
filter_40 = csf.columns.get_level_values(0).str.contains("TFSi_40") 
filter_60 = csf.columns.get_level_values(0).str.contains("TFSi_60")
filter_80 = csf.columns.get_level_values(0).str.contains("TFSi_80")

filter_bad = csf.columns.get_level_values(0).isin(["20200406_TFSi_40_2_02_cc", "20200406_TFSi_60_5_01_cc"])
filter_charge = csf.columns.get_level_values(1) == "Charge_Capacity"
filter_charge_mah = csf.columns.get_level_values(1) == "Charge_Capacity(mAh/g)"

# filter out the charge capacity for the three first cycles exluding the bad cells:
scsf = csf.loc[1:3, filter_charge_mah & ~filter_bad]

# plotting
scsf.columns = scsf.columns.droplevel(1)
scsf.hvplot.scatter(ylim=(0, 5000))

```

### Example: Plot all cycles

```python
voltage_curves = dict()
for label in b.experiment.cell_names:
    d = b.experiment.data[label]
    curves = d.get_cap(label_cycle_number=True, interpolated=True, number_of_points=100)
    
    if not curves.empty:
        curve = hv.Curve(curves, kdims=["capacity", "cycle"],       
                         vdims="voltage").groupby("cycle").overlay()#.opts(show_legend=False)
        voltage_curves[label] = curve
    else:
        print(f"[{label} empty]")
        
NdLayout_voltage_curves = hv.NdLayout(voltage_curves, kdims='label').cols(3)

palette = 'Spectral'
NdLayout_voltage_curves.opts(
    hv.opts.Curve(color=hv.Palette(palette), tools=['hover']),
    hv.opts.NdOverlay(shared_axes=False),
    hv.opts.NdLayout(),
)
```